In [271]:
corrections_base_directory = "../../epi_corrections_out_2019_07_02_tumor_exluded_from_rcbv"
neuromorphometrics_file = corrections_base_directory + "/labels_Neuromorphometrics.nii"
neuromorphometrics_xml_file = corrections_base_directory + "/labels_Neuromorphometrics.xml"

In [272]:
%run utils.py
%run visualization.py

Load data

In [273]:
nm_data, nm_pixdim, nm_header = load_nifti(neuromorphometrics_file)

Visualize

In [275]:
#%gui qt5

In [276]:
#spimagine_show_mni_volume_numpy(nm_data, stackUnits=nm_pixdim)

Count number of regions

In [277]:
region_values = np.unique(nm_data)

Get region names from the xml file and remove region values from region_values that are not found in the xml file. This is only non-brain region if original Neuromorphometrics file is used

In [278]:
import xml.etree.ElementTree as ET
tree = ET.parse(neuromorphometrics_xml_file)
region_names = []
region_values_temp = []
for i in tree.iter():
    index = i.find("index")
    if index != None:
        index_int = np.int(index.text)
        if index_int in region_values:
            name = i.find("name")
            region_values_temp += [index_int]
            region_names += [name.text]
region_values = np.array(region_values_temp) # Update region_values. Removes 0 (outside of the brain) region from region_values, since it does not have a name
region_names = np.array(region_names)

In [279]:
#region_names

Verify that we have the same amount of regions with name and values

In [280]:
region_values.shape == region_names.shape

True

Create empty array to place the merged data

In [281]:
nm_data_lrmerged = np.zeros_like(nm_data)

Create empty array to place the merged names

In [282]:
region_names_merged = []

Perform the merge

In [283]:
value = 1
for name in region_names:
    if "Left" in name:
        left_name = name
        right_name = region_names[region_names == "Right" + name[len("Left"):]][0]
        left_name_value = region_values[np.where(region_names == left_name)][0]
        right_name_value = region_values[np.where(region_names == right_name)][0]
        """
        print(left_name)
        print(left_name_value)
        print(right_name)
        print(right_name_value)
        """
        # Set left and right regions to the value of the left region,
        # thereby merging left and right region into one region.
        #nm_data_lrmerged[nm_data == left_name_value] = left_name_value
        #nm_data_lrmerged[nm_data == right_name_value] = left_name_value
        
        nm_data_lrmerged[nm_data == left_name_value] = value
        nm_data_lrmerged[nm_data == right_name_value] = value
        
        # Append region names list
        region_names_merged += ["Left & right" + name[len("Left"):]]
        
        # Increment value
        value += 1
    elif not "Right" in name:
        name_value = region_values[np.where(region_names == name)][0]
        #nm_data_lrmerged[nm_data == name_value] = name_value
        nm_data_lrmerged[nm_data == name_value] = value
        region_names_merged += [name]
        
        # Incremnt value
        value += 1
region_names_merged = np.array(region_names_merged)

Visualize the merged volume

In [284]:
#spimagine_show_mni_volume_numpy(nm_data_lrmerged)

Verify that we have the same amount of regions with merged name and values

In [285]:
region_names_merged.shape == np.unique(nm_data_lrmerged)[1:].shape

True

In [296]:
np.unique(nm_data_lrmerged)[1:]

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26.,
       27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39.,
       40., 41., 42., 43., 44., 45., 46., 47., 48., 49., 50., 51., 52.,
       53., 54., 55., 56., 57., 58., 59., 60., 61., 62., 63., 64., 65.,
       66., 67., 68., 69., 70., 71., 72.])

In [300]:
region_names_merged

array(['3rd Ventricle', '4th Ventricle', 'Left & right Accumbens Area',
       'Left & right Amygdala', 'Brain Stem', 'Left & right Caudate',
       'Left & right Cerebellum Exterior',
       'Left & right Cerebellum White Matter',
       'Left & right Cerebral White Matter', 'CSF',
       'Left & right Hippocampus', 'Left & right Inf Lat Vent',
       'Left & right Lateral Ventricle', 'Left & right Pallidum',
       'Left & right Putamen', 'Left & right Thalamus Proper',
       'Left & right Ventral DC', 'Left & right vessel', 'Optic Chiasm',
       'Cerebellar Vermal Lobules I-V',
       'Cerebellar Vermal Lobules VI-VII',
       'Cerebellar Vermal Lobules VIII-X', 'Left & right Basal Forebrain',
       'Left & right ACgG anterior cingulate gyrus',
       'Left & right AIns anterior insula',
       'Left & right AOrG anterior orbital gyrus',
       'Left & right AnG angular gyrus',
       'Left & right Calc calcarine cortex',
       'Left & right CO central operculum', 'Left & right 

In [287]:
print("Went from %i to %i regions by merging left and right" % (region_names.shape[0], region_names_merged.shape[0]))

Went from 136 to 72 regions by merging left and right


Save merged file using existing header and affine information

In [288]:
# Need affine information
nm_affine = nib.load(neuromorphometrics_file).affine

# Create Nifti image class instance
#nm_lrmerged_img = nib.spatialimages.SpatialImage(nm_data_lrmerged, affine=nm_affine, header=nm_header)
nm_lrmerged_img = nib.Nifti1Image(nm_data_lrmerged.astype(np.uint8), nm_affine, header=nm_header)

# Save cbv by overwriting original cbv file
nib.save(nm_lrmerged_img, corrections_base_directory + "/labels_Neuromorphometrics_lrmerged.nii")

In [289]:
# https://stackoverflow.com/questions/44397617/change-data-type-in-numpy-and-nibabel
#nm_header['sizeof_hdr']

In [290]:
#image.get_data_dtype()